In [7]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import src.aformer_atac_cage as aformer
#import src.aformer_TF as aformer
from src.layers.layers import *
import src.metrics as metrics
from src.optimizers import *
import src.schedulers as schedulers
import src.utils as utils

import training_utils_atac_cage as training_utils


from scipy import stats

In [9]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-5')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    #options.experimental_threading.max_intra_op_parallelism = 1
    mixed_precision.set_global_policy('mixed_bfloat16')
    tf.config.optimizer.set_jit(True)
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 1
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS

2023-04-27 23:14:39.956831: E tensorflow/core/common_runtime/eager/context.cc:575] Unable to close remote context with ID 13174737909895361916 for worker: /job:worker/replica:0/task:0 due to failed to connect to all addresses
Additional GRPC error information from remote target /job:worker/replica:0/task:0:
:{"created":"@1682637279.956459071","description":"Failed to pick subchannel","file":"external/com_github_grpc_grpc/src/core/ext/filters/client_channel/client_channel.cc","file_line":3940,"referenced_errors":[{"created":"@1682637279.956455463","description":"failed to connect to all addresses","file":"external/com_github_grpc_grpc/src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":392,"grpc_status":14}]}
2023-04-27 23:14:39.957096: W tensorflow/core/common_runtime/eager/context.cc:1607] Unable to destroy server_ object, so releasing instead. Servers don't support clean shutdown.
2023-04-27 23:14:39.963198: I tensorflow/core/distributed_runtime/rpc/gr

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: node-5


INFO:tensorflow:Initializing the TPU system: node-5


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [10]:
g = tf.random.Generator.from_seed(1)
with strategy.scope():

    tr_data_it,val_data_it,val_data_ho_it, val_data_TSS_it,val_data_TSS_ho_it = \
        training_utils.return_distributed_iterators("gs://picard-testing-176520/paired_rampage_atac/262k/genformer_atac_rampage_globalacc_conv_rpgc_5prime",
                                     "gs://picard-testing-176520/paired_rampage_atac/262k/genformer_atac_rampage_globalacc_conv_rpgc_5prime_holdout",
                                     "gs://picard-testing-176520/paired_rampage_atac/262k/genformer_atac_rampage_globalacc_conv_rpgc_TSS_5prime",
                                     "gs://picard-testing-176520/paired_rampage_atac/262k/genformer_atac_rampage_globalacc_conv_rpgc_TSS_5prime_holdout",
                                     8,
                                     262144,
                                     10,
                                     65536,
                                     2048,
                                     256,
                                     128,
                                     4,
                                     5,
                                     strategy,
                                     options,
                                     0.025,
                                     1024,
                                     False,
                                     True,
                                     True,
                                     False,
                                     g)



In [11]:
with strategy.scope():


    model = aformer.aformer(kernel_transformation = 'relu_kernel_transformation',
                             dropout_rate = 0.30,
                             pointwise_dropout_rate=0.1,
                             input_length = 262144,
                             output_length = 2048,
                             final_output_length = 1536,
                             num_heads = 8,
                             numerical_stabilizer = 0.001,
                             nb_random_features = 256,
                             num_transformer_layers = 6,
                             norm=True,
                             BN_momentum=0.90,
                             max_seq_length = 2048,
                             use_rot_emb = True,
                             use_mask_pos = False, 
                             normalize = True,
                             seed = 3,
                             load_init=False,
                             stable_variant=False,
                             inits=None,
                             freeze_conv_layers=False,
                             freeze_BN_layers=False,
                             predict_atac=False,
                             learnable_PE=True,
                             filter_list_seq=[768,896,1024,1152,1280,1536])
    
    def build_step(iterator): #input_batch, model, optimizer, organism, gradient_clip):
        @tf.function(jit_compile=True)
        def test_step(inputs):
            sequence=tf.cast(inputs['sequence'],dtype=tf.bfloat16)
            atac=tf.cast(inputs['atac'],dtype=tf.bfloat16)
            target=tf.cast(inputs['target'],dtype=tf.float32)
            #global_acc=tf.cast(inputs['global_acc'],dtype=tf.bfloat16)         
            input_tuple = sequence,atac#,global_acc

            output = model(input_tuple,
                           training=False)

        for _ in tf.range(1): ## for loop within @tf.fuction for improved TPU performance
            strategy.run(test_step, args=(next(iterator),))
            
            
    build_step(val_data_it)

    print('ran test input')
    #model.load_weights("gs://picard-testing-176520/genformer_atac_pretrain/models/aformer_hg_262k_load-True_LR-0.01_T-7_D-0.3_2023-04-13_15:39:43/final/saved_model")
    model.load_weights("gs://picard-testing-176520/genformer_atac_pretrain/models/aformer_hg_262k_load-True_LR-0.01_T-6_D-0.3_2023-04-26_00:40:44/iteration_84/saved_model")
    print('loaded weights')

/opt/conda/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


ran test input
loaded weights


In [5]:
with strategy.scope():
    metric_dict = {}
    """
    optimizer1 = tf.keras.optimizers.Adam(learning_rate=1.0e-07,
                                          epsilon=1.0e-08)
    optimizer2 = tf.keras.optimizers.Adam(learning_rate=1.0e-07,
                                          epsilon=1.0e-08)
    optimizer3 = tf.keras.optimizers.Adam(learning_rate=1.0e-04,
                                          epsilon=1.0e-08)
    """
    optimizer1 = tfa.optimizers.AdaBelief(
        learning_rate= 1.0e-08, 
        epsilon=1.0e-14,
        rectify=True)
    optimizer2 = tfa.optimizers.AdaBelief(
        learning_rate= 1.0e-08, 
        epsilon=1.0e-14,
        rectify=True)
    optimizer3 = tfa.optimizers.AdaBelief(
        learning_rate= 1.0e-04, 
        epsilon=1.0e-14,
        rectify=True)
    optimizers_in=optimizer1,optimizer2,optimizer3

    loss_fn = tf.keras.losses.Poisson(reduction=tf.keras.losses.Reduction.NONE)
    
    metric_dict["train_loss"] = tf.keras.metrics.Mean("train_loss",
                                                 dtype=tf.float32)
    
    def dist_train_step_cage_only(iterator):    
        @tf.function(jit_compile=True)
        def train_step(inputs):
            sequence=tf.cast(inputs['sequence'],dtype=tf.bfloat16)
            atac=tf.cast(inputs['atac'],dtype=tf.bfloat16)
            target=tf.cast(inputs['target'],dtype=tf.float32)
            mask=tf.cast(inputs['mask'],dtype=tf.float32)
            mask_gathered=tf.cast(inputs['mask_gathered'],dtype=tf.int32)
            peaks=tf.cast(inputs['peaks'],dtype=tf.int32)
            atac_orig=tf.cast(inputs['atac_orig'],dtype=tf.bfloat16)
            tss_tokens=tf.cast(inputs['tss_tokens'],dtype=tf.int32)
            

            input_tuple = sequence, atac

            with tf.GradientTape() as tape:

                conv_vars = model.stem_conv.trainable_variables + \
                            model.stem_res_conv.trainable_variables + \
                            model.stem_pool.trainable_variables + \
                            model.conv_tower.trainable_variables + \
                            model.stem_conv_atac.trainable_variables + \
                                model.stem_res_conv_atac.trainable_variables + \
                                        model.stem_pool_atac.trainable_variables + \
                                            model.conv_tower_atac.trainable_variables
                
                performer_vars =model.pos_embedding_learned.trainable_variables + \
                                    model.performer.trainable_variables + \
                                        model.final_pointwise_conv.trainable_variables
                                                

                heads_vars = model.final_dense_profile_FT.trainable_variables 

                vars_all = conv_vars + performer_vars + heads_vars

                output_profile = model(input_tuple,
                                       training=True)

                output_profile = tf.cast(output_profile,dtype=tf.float32)
                

                cage_loss = tf.reduce_mean(loss_fn(target[:,:,:],
                                                             output_profile[:,:,:])) * (1. / 24)
                loss = cage_loss
            

            gradients = tape.gradient(loss, vars_all)
            """
            gradients, _ = tf.clip_by_global_norm(gradients, 
                                                  0.2)

            optimizer1.apply_gradients(zip(gradients[:len(conv_vars)], 
                                           conv_vars))
            optimizer2.apply_gradients(zip(gradients[len(conv_vars):len(conv_vars+pointwise_vars)], 
                                           pointwise_vars))
            optimizer3.apply_gradients(zip(gradients[len(conv_vars+pointwise_vars):], 
                                           heads_vars))
            metric_dict["train_loss"].update_state(loss)
            """
            return gradients,loss,output_profile,sequence,atac,target,atac_orig,mask,tss_tokens

        for _ in tf.range(1):
            gradients_out,loss,output_profile,sequence,atac,target,atac_orig,mask,tss_tokens = strategy.run(train_step,
                         args=(next(iterator),))
        return gradients_out,loss,output_profile,sequence,atac,target,atac_orig,mask,tss_tokens


In [6]:
from scipy.stats.stats import pearsonr  
with strategy.scope():
    def sum_log(x):
        return np.log10(1.0 + np.nansum(x))
    
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(2, 10):
        start = time.time()
        if epoch_i == 1:
            build_step(tr_data_it)
            total_params = 0
            for k in model.trainable_variables:
                var = k.values[0]
                total_params += tf.size(var)
            print('total params: ' + str(total_params))
        

        grads_out,loss,output_profile,sequence,atac,target,atac_orig,mask,tss_tokens = dist_train_step_cage_only(tr_data_it)
        
        print([tf.reduce_sum(tf.cast(tf.math.is_nan(x.values[0]),dtype=tf.float32)) for x in grads_out])
        print(loss)
        #train_step(data_tr)
        
        print('hg_train_loss: ' + str(metric_dict['train_loss'].result().numpy()))
        break
        #continue
        '''
        val_step(data_val)
        #val_step(data_val)

        val_losses.append(metric_dict['val_loss'].result().numpy())
        print('hg_val_loss: ' + str(metric_dict['val_loss'].result().numpy()))
        #print('hg_val_loss_CAGE: ' + str(metric_dict['val_loss_CAGE'].result().numpy()))
        #print('hg_val_loss_ATAC: ' + str(metric_dict['val_loss_ATAC'].result().numpy()))
        #print('hg_val_cage_pearson: ' + str(metric_dict['CAGE_PearsonR'].result()['PearsonR'].numpy()))
        #print('hg_val_cage_R2: ' + str(metric_dict['CAGE_R2'].result()['R2'].numpy()))
        
        print('hg_val_atac_pearson: ' + str(metric_dict['ATAC_PearsonR'].result()['PearsonR'].numpy()))
        print('hg_val_atac_R2: ' + str(metric_dict['ATAC_R2'].result()['R2'].numpy()))
        
        
        #print('hg_val_atac_baseline_pearson: ' + str(metric_dict['ATAC_PearsonR_baseline'].result()['PearsonR'].numpy()))
        #print('hg_val_atac_baseline_R2: ' + str(metric_dict['ATAC_R2_baseline'].result()['R2'].numpy()))
        """
        val_step_TSS(data_val_TSS)
        #val_step_TSS(data_val_TSS)
        
        y_trues = metric_dict['corr_stats'].result()['y_trues'].numpy()
        y_preds = metric_dict['corr_stats'].result()['y_preds'].numpy()
        cell_types = metric_dict['corr_stats'].result()['cell_types'].numpy()
        gene_map = metric_dict['corr_stats'].result()['gene_map'].numpy()


        figures, corrs_overall = training_utils.make_plots(y_trues,
                                           y_preds, 
                                           cell_types, 
                                           gene_map,50)
        
        cell_spec,gene_spec,cell_spec_sp,gene_spec_sp = corrs_overall
        
        print('cell spec corr: ' + str(cell_spec))
        print('gene_spec_corr:' + str(gene_spec))
        print('cell spec corr_sp: ' + str(cell_spec_sp))
        print('gene_spec_corr_sp:' + str(gene_spec_sp))
        """
        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('duration(mins): ' + str(duration))
        print('patience counter at: ' + str(patience_counter))
        
        for key, item in metric_dict.items():
            item.reset_state()
        '''

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: shape=(

In [11]:
tf.reduce_max(tss_tokens.values[0])

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [17]:
inside_checkpoint

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/batch_norm/beta/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/batch_norm/gamma/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/batch_norm/moving_mean/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-0/batch_norm/moving_variance/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-1/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-0/layer_with_weights-1/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [5, 768, 768]),
 ('conv_tower/layer_with_weights-0/layer_with_weights-1/_layer/layer_with_weights-0/batch_norm/beta/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('conv_tower/layer_wit

In [30]:
checkpoint_path="gs://picard-testing-176520/genformer_atac_pretrain/models/aformer_hg_262k_load-True_LR-0.01_T-6_D-0.3_2023-04-26_00:40:44/iteration_84"
inside_checkpoint=tf.train.list_variables(tf.train.latest_checkpoint(checkpoint_path))
reader = tf.train.load_checkpoint(checkpoint_path)
reader.get_tensor('performer/layers/2/FFN/FFN_layer_norm/layer_norm/beta/.ATTRIBUTES/VARIABLE_VALUE')
#print(reader.get_tensor('performer/layers/2/FFN/FFN_layer_norm/layer_norm/beta/.ATTRIBUTES/VARIABLE_VALUE').shape)

array([-0.11682089, -0.00745887, -0.00873533, ...,  0.00139148,
       -0.01388337, -0.01154972], dtype=float32)

In [31]:
#for i,k in enumerate(model.performer.trainable_variables):
#    print(i,k.name,k.shape)
model.performer.trainable_variables[31]


TPUDistributedVariable:{
  0: <tf.Variable 'aformer/shared_transformer/transformer_layer/FFN/layer_norm_fp32/layer_normalization_5/beta:0' shape=(1600,) dtype=float32, numpy=
array([-0.11682089, -0.00745887, -0.00873533, ...,  0.00139148,
       -0.01388337, -0.01154972], dtype=float32)>,
  1: <tf.Variable 'aformer/shared_transformer/transformer_layer/FFN/layer_norm_fp32/layer_normalization_5/beta/replica_1:0' shape=(1600,) dtype=float32, numpy=
array([-0.11682089, -0.00745887, -0.00873533, ...,  0.00139148,
       -0.01388337, -0.01154972], dtype=float32)>,
  2: <tf.Variable 'aformer/shared_transformer/transformer_layer/FFN/layer_norm_fp32/layer_normalization_5/beta/replica_2:0' shape=(1600,) dtype=float32, numpy=
array([-0.11682089, -0.00745887, -0.00873533, ...,  0.00139148,
       -0.01388337, -0.01154972], dtype=float32)>,
  3: <tf.Variable 'aformer/shared_transformer/transformer_layer/FFN/layer_norm_fp32/layer_normalization_5/beta/replica_3:0' shape=(1600,) dtype=float32, numpy=
